In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [3]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [4]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [5]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [6]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [7]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df

import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [8]:
def multi_columns_merging(sg, sg_hl):
    a_df = load_to_df( sg )
    b_df = load_to_df( sg_hl )
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    all_df2 = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df2.dropna(axis=0)

col = ['요일']
target=['연령대','시간대']
def drop_filter(df1,target):
    df_a=df1.drop(df1.filter(regex='|'.join(target)),axis=1)
    return df_a 

def multi_merge(a_df, b_df):
    all_df = pd.merge(left=a_df, right=b_df, how='left', on=['기준_년_분기_코드','상권_코드'], sort=False)
    all_df = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df.dropna(axis=0)

def multi_columns_merging2(df, sg):
    a_df = load_to_df( sg )
    all_df = pd.merge(left=df, right=a_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    all_df2 = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df2.dropna(axis=0)

In [10]:
df0=load_to_df( '상권-영역' )
# df0.head()
# df0.info()
df10 = load_to_df('행정동별 상권변화지표')
# df10.head()
df15=load_to_df( '자치구별 상권변화지표' )
# df15.info()
# df15.head()

ValueError: Length mismatch: Expected axis has 10 elements, new values have 8 elements

In [15]:
df10_1=df10[['행정동_코드','행정동_코드_명']]
df15_1=df15[["시군구_코드",'시군구_코드_명']]

In [16]:
map_df1=df0.merge(df15_1, on='시군구_코드').drop_duplicates()
map_df=map_df1.merge(df10_1, on='행정동_코드').drop_duplicates()

In [13]:
map_df

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,시군구_코드_명,행정동_코드_명
0,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,관악구,낙성대동
25,A,골목상권,1000799,남부순환로234길,196264,442001,11620,11620585,관악구,낙성대동
50,A,골목상권,1000780,관악로10길,195952,441919,11620,11620585,관악구,낙성대동
75,A,골목상권,1000781,관악로14길,195965,442143,11620,11620585,관악구,낙성대동
100,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,관악구,청룡동
...,...,...,...,...,...,...,...,...,...,...
38082,A,골목상권,1000454,통일로37길,195100,454161,11410,11410620,서대문구,홍제1동
38107,A,골목상권,1000455,통일로39길,194935,454243,11410,11410620,서대문구,홍제1동
38132,A,골목상권,1000457,홍제내길,194424,454124,11410,11410620,서대문구,홍제1동
38157,D,발달상권,1001185,서울 서대문구 신촌역,194426,450896,11410,11410585,서대문구,신촌동


In [14]:
# map_df.to_csv('map.csv' , index=False )
# s1.to_csv('sales.csv', index=False)

In [13]:
# load_to_df( '상권-상권변화지표' )

In [14]:
a_df = multi_columns_merging('상권배후지-추정유동인구','상권배후지-상주인구' )
# a_df.info()
# a_df  #2020

In [15]:
target3=['월요','화요','수요','목요','금요']
target4=['토요','일요']
def add_week2(df,target3,target4):
    df['hl_주중_유동인구_수']=df.filter(regex='|'.join(target3)).sum(axis=1)
    df['hl_주말_유동인구_수']=df.filter(regex='|'.join(target4)).sum(axis=1)
    df_a=df.drop(df.filter(regex='|'.join(target3+target4)),axis=1)
    return df_a

In [16]:
b_df=add_week2(a_df,target3,target4)
# b_df

In [17]:
c_df=multi_columns_merging2(b_df, '상권배후지-직장인구')    # 유동, 상주, 직장
c_df.info()  #2020

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020 entries, 0 to 2019
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   기준_년_분기_코드     2020 non-null   object
 1   상권_코드          2020 non-null   object
 2   hl_총_유동인구_수    2020 non-null   int64 
 3   hl_남성_유동인구_수   2020 non-null   int64 
 4   hl_여성_유동인구_수   2020 non-null   int64 
 5   hl_총_상주인구_수    2020 non-null   int64 
 6   hl_남성_상주인구_수   2020 non-null   int64 
 7   hl_여성_상주인구_수   2020 non-null   int64 
 8   hl_총_가구_수      2020 non-null   int64 
 9   hl_아파트_가구_수    2020 non-null   int64 
 10  hl_비_아파트_가구_수  2020 non-null   int64 
 11  hl_주중_유동인구_수   2020 non-null   int64 
 12  hl_주말_유동인구_수   2020 non-null   int64 
 13  hl_총_직장_인구_수   2020 non-null   int64 
 14  hl_남성_직장_인구_수  2020 non-null   int64 
 15  hl_여성_직장_인구_수  2020 non-null   int64 
dtypes: int64(14), object(2)
memory usage: 268.3+ KB


In [18]:
c_df=c_df.rename(columns={'hl_총_직장_인구_수':'hl_총_직장인구_수','hl_남성_직장_인구_수':'hl_남성_직장인구_수','hl_여성_직장_인구_수':'hl_여성_직장인구_수'})
c_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020 entries, 0 to 2019
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   기준_년_분기_코드     2020 non-null   object
 1   상권_코드          2020 non-null   object
 2   hl_총_유동인구_수    2020 non-null   int64 
 3   hl_남성_유동인구_수   2020 non-null   int64 
 4   hl_여성_유동인구_수   2020 non-null   int64 
 5   hl_총_상주인구_수    2020 non-null   int64 
 6   hl_남성_상주인구_수   2020 non-null   int64 
 7   hl_여성_상주인구_수   2020 non-null   int64 
 8   hl_총_가구_수      2020 non-null   int64 
 9   hl_아파트_가구_수    2020 non-null   int64 
 10  hl_비_아파트_가구_수  2020 non-null   int64 
 11  hl_주중_유동인구_수   2020 non-null   int64 
 12  hl_주말_유동인구_수   2020 non-null   int64 
 13  hl_총_직장인구_수    2020 non-null   int64 
 14  hl_남성_직장인구_수   2020 non-null   int64 
 15  hl_여성_직장인구_수   2020 non-null   int64 
dtypes: int64(14), object(2)
memory usage: 268.3+ KB


In [19]:
hl_people_df=c_df.merge(map_df, on=['상권_코드']).drop_duplicates()

In [20]:
hl_people_df

,기준_년_분기_코드,상권_코드,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,hl_총_상주인구_수,hl_남성_상주인구_수,hl_여성_상주인구_수,hl_총_가구_수,hl_아파트_가구_수,...,hl_여성_직장인구_수,상권_구분_코드,상권_구분_코드_명,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,시군구_코드_명,행정동_코드_명
0,2019_1,1000645,161694,87243,74446,14242,7069,7173,5547,191,...,788,A,골목상권,부일로11길,185004,444272,11530,11530790,구로구,수궁동
1,2019_2,1000645,265306,132335,132964,14242,7069,7173,5547,191,...,662,A,골목상권,부일로11길,185004,444272,11530,11530790,구로구,수궁동
2,2019_1,1000646,190850,108281,82567,11001,5438,5563,4425,178,...,500,A,골목상권,부일로15길,185268,443877,11530,11530790,구로구,수궁동
3,2019_2,1000646,288272,155795,132476,11001,5438,5563,4425,178,...,535,A,골목상권,부일로15길,185268,443877,11530,11530790,구로구,수궁동
4,2019_1,1000647,238761,118531,120230,8171,4124,4047,3163,939,...,647,A,골목상권,부일로1길,184095,443791,11530,11530790,구로구,수궁동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019_2,1000804,1707550,850398,857157,21414,10586,10828,11285,1068,...,1261,A,골목상권,당곡길,193666,443497,11620,11620525,관악구,보라매동
2016,2019_1,1000805,1107208,688379,418824,15749,9587,6162,10033,399,...,302,A,골목상권,대학14길,194319,440645,11620,11620735,관악구,대학동
2017,2019_2,1000805,1351332,834454,516874,15749,9587,6162,10033,399,...,473,A,골목상권,대학14길,194319,440645,11620,11620735,관악구,대학동
2018,2019_1,1000806,2077224,1291806,785416,17601,10842,6759,12309,360,...,646,A,골목상권,대학길,194450,440913,11620,11620735,관악구,대학동


In [21]:
# 비율 구하기
def ratio(df,col1,col2):
    total = df[col1]+df[col2]
    total2 = df[col1]/total*100
    return total2

In [22]:
# hl_people_df['hl_남성_유동인구_비율']=ratio(hl_people_df,'hl_남성_유동인구_수', 'hl_여성_유동인구_수')
# hl_people_df['hl_여성_유동인구_비율']=ratio(hl_people_df,'hl_여성_유동인구_수', 'hl_남성_유동인구_수')
# hl_people_df['hl_주중_유동인구_비율']=ratio(hl_people_df,'hl_주중_유동인구_수', 'hl_주말_유동인구_수')
# hl_people_df['hl_주말_유동인구_비율']=ratio(hl_people_df,'hl_주말_유동인구_수', 'hl_주중_유동인구_수')

# hl_people_df['hl_남성_상주인구_비율']=ratio(hl_people_df,'hl_남성_상주인구_수', 'hl_여성_상주인구_수')
# hl_people_df['hl_여성_상주인구_비율']=ratio(hl_people_df,'hl_여성_상주인구_수', 'hl_남성_상주인구_수')

# hl_people_df['hl_남성_직장인구_비율']=ratio(hl_people_df,'hl_남성_직장_인구_수', 'hl_여성_직장_인구_수')
# hl_people_df['hl_여성_직장인구_비율']=ratio(hl_people_df,'hl_여성_직장_인구_수', 'hl_남성_직장_인구_수')

In [23]:
hl_people_df.columns

Index(['기준_년_분기_코드', '상권_코드', 'hl_총_유동인구_수', 'hl_남성_유동인구_수', 'hl_여성_유동인구_수',
       'hl_총_상주인구_수', 'hl_남성_상주인구_수', 'hl_여성_상주인구_수', 'hl_총_가구_수',
       'hl_아파트_가구_수', 'hl_비_아파트_가구_수', 'hl_주중_유동인구_수', 'hl_주말_유동인구_수',
       'hl_총_직장인구_수', 'hl_남성_직장인구_수', 'hl_여성_직장인구_수', '상권_구분_코드', '상권_구분_코드_명',
       '상권_코드_명', '엑스좌표_값', '와이좌표_값', '시군구_코드', '행정동_코드', '시군구_코드_명',
       '행정동_코드_명'],
      dtype='object')

In [24]:
hl_people_data=hl_people_df[['기준_년_분기_코드','상권_구분_코드_명','상권_코드_명','시군구_코드_명', '행정동_코드_명',
                             'hl_총_유동인구_수', 'hl_총_상주인구_수','hl_총_직장인구_수',
                             'hl_주중_유동인구_수','hl_주말_유동인구_수',
                             'hl_남성_유동인구_수', 'hl_여성_유동인구_수',
                             'hl_남성_상주인구_수', 'hl_여성_상주인구_수',
                             'hl_남성_직장인구_수', 'hl_여성_직장인구_수' ]]

In [25]:
hl_people_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020 entries, 0 to 2019
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   기준_년_분기_코드    2020 non-null   object
 1   상권_구분_코드_명    2020 non-null   object
 2   상권_코드_명       2020 non-null   object
 3   시군구_코드_명      2020 non-null   object
 4   행정동_코드_명      2020 non-null   object
 5   hl_총_유동인구_수   2020 non-null   int64 
 6   hl_총_상주인구_수   2020 non-null   int64 
 7   hl_총_직장인구_수   2020 non-null   int64 
 8   hl_주중_유동인구_수  2020 non-null   int64 
 9   hl_주말_유동인구_수  2020 non-null   int64 
 10  hl_남성_유동인구_수  2020 non-null   int64 
 11  hl_여성_유동인구_수  2020 non-null   int64 
 12  hl_남성_상주인구_수  2020 non-null   int64 
 13  hl_여성_상주인구_수  2020 non-null   int64 
 14  hl_남성_직장인구_수  2020 non-null   int64 
 15  hl_여성_직장인구_수  2020 non-null   int64 
dtypes: int64(11), object(5)
memory usage: 268.3+ KB


In [26]:
hl_people_data_en=hl_people_data.rename(columns={'기준_년_분기_코드':'yesrSe','상권_구분_코드_명':'sgSeCdName', '상권_코드_명':'sgCdName','시군구_코드_명':'guName', '행정동_코드_명':'dongName',
                                              'hl_총_유동인구_수':'hlTotalFlowPop', 'hl_총_상주인구_수':'hlTotalStayPop','hl_총_직장인구_수':'hlTotalOffPop', 
                                              'hl_남성_유동인구_수':'hlMenFlowCnt', 'hl_여성_유동인구_수':'hlWomenFlowCnt', 
                                              'hl_주중_유동인구_수':'hlWeekdayFlowCnt','hl_주말_유동인구_수':'hlWeekendFlowCnt',
                                              'hl_남성_상주인구_수':'hlMenStayCnt', 'hl_여성_상주인구_수':'hlWomenStayCnt', 
                                              'hl_남성_직장인구_수':'hlMenOffCnt','hl_여성_직장인구_수':'hlWomenOffCnt'})

In [27]:
# hl_people_data_en.info()

In [28]:
hl_people_data_en=hl_people_data_en.replace('2019_2', '2019년2분기').replace('2019_1','2019년1분기')
hl_people_data_en

,yesrSe,sgSeCdName,sgCdName,guName,dongName,hlTotalFlowPop,hlTotalStayPop,hlTotalOffPop,hlWeekdayFlowCnt,hlWeekendFlowCnt,hlMenFlowCnt,hlWomenFlowCnt,hlMenStayCnt,hlWomenStayCnt,hlMenOffCnt,hlWomenOffCnt
0,2019년1분기,골목상권,부일로11길,구로구,수궁동,161694,14242,1723,110987,50715,87243,74446,7069,7173,935,788
1,2019년2분기,골목상권,부일로11길,구로구,수궁동,265306,14242,1736,178241,87052,132335,132964,7069,7173,1074,662
2,2019년1분기,골목상권,부일로15길,구로구,수궁동,190850,11001,1558,123714,67141,108281,82567,5438,5563,1058,500
3,2019년2분기,골목상권,부일로15길,구로구,수궁동,288272,11001,1369,184026,104237,155795,132476,5438,5563,834,535
4,2019년1분기,골목상권,부일로1길,구로구,수궁동,238761,8171,2231,157045,81719,118531,120230,4124,4047,1584,647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019년2분기,골목상권,당곡길,관악구,보라매동,1707550,21414,2870,1207331,500246,850398,857157,10586,10828,1609,1261
2016,2019년1분기,골목상권,대학14길,관악구,대학동,1107208,15749,610,771817,335399,688379,418824,9587,6162,308,302
2017,2019년2분기,골목상권,대학14길,관악구,대학동,1351332,15749,1117,938324,412996,834454,516874,9587,6162,644,473
2018,2019년1분기,골목상권,대학길,관악구,대학동,2077224,17601,1794,1456614,620603,1291806,785416,10842,6759,1148,646


In [29]:
hl_people_data_en.to_csv('hl_people_data_en.csv', index=False)

In [30]:
df4=load_to_df('상권배후지-집객시설')           #2020
# df4

In [31]:
d_df=c_df.merge(df4, on=['기준_년_분기_코드','상권_코드']).drop_duplicates()
# d_df        # 유동, 상주, 직장 , 집객   2020

In [32]:
df7=load_to_df('상권배후지-아파트')
df7.head()

df7=df7.drop(df7.filter(regex='아파트_면적_|아파트_가격_'),axis=1).drop_duplicates()
# df7 #2018

In [33]:
e_df=d_df.merge(df7, on=['기준_년_분기_코드','상권_코드'], how='outer').fillna('0')
# e_df.info()    # 유동, 상주, 직장 , 집객 , 아파트  2020

e_df   # 유동, 상주, 직장 , 집객 , 아파트  2020

In [34]:
# ###########

In [35]:
df19=load_to_df('상권배후지-추정매출')
df8=load_to_df('상권배후지-점포')

In [36]:
df19=df19.drop(df19.filter(regex='시간대|연령대|요일'),axis=1).drop_duplicates()
df19.columns   , df8.columns
df8 = df8.rename(columns={'hl_점포_수':'hl_점포수'})

In [37]:
df8_19=df19.merge(df8, on=['기준_년_분기_코드','상권_코드','서비스_업종_코드','서비스_업종_코드_명']).drop_duplicates()

In [38]:
df8_19 = df8_19.drop(df8_19.filter(regex='|'.join('연령대,시간대,요일')),axis=1)

In [39]:
hl_store_sales=df8_19.merge(map_df, on='상권_코드').drop_duplicates()

In [40]:
hl_store_sales.columns

Index(['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '서비스_업종_코드_명', 'hl_점포수_x',
       'hl_당월_매출_금액', 'hl_당월_매출_건수', 'hl_주중_매출_비율', 'hl_주말_매출_비율',
       'hl_남성_매출_비율', 'hl_여성_매출_비율', 'hl_주중_매출_금액', 'hl_주말_매출_금액',
       'hl_남성_매출_금액', 'hl_여성_매출_금액', 'hl_주중_매출_건수', 'hl_주말_매출_건수',
       'hl_남성_매출_건수', 'hl_여성_매출_건수', 'hl_점포수_y', 'hl_유사_업종_점포_수', 'hl_개업_율',
       'hl_개업_점포_수', 'hl_폐업_률', 'hl_폐업_점포_수', 'hl_프랜차이즈_점포_수', '상권_구분_코드',
       '상권_구분_코드_명', '상권_코드_명', '엑스좌표_값', '와이좌표_값', '시군구_코드', '행정동_코드',
       '시군구_코드_명', '행정동_코드_명'],
      dtype='object')

In [41]:
map_df

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,시군구_코드_명,행정동_코드_명
0,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,관악구,낙성대동
25,A,골목상권,1000799,남부순환로234길,196264,442001,11620,11620585,관악구,낙성대동
50,A,골목상권,1000780,관악로10길,195952,441919,11620,11620585,관악구,낙성대동
75,A,골목상권,1000781,관악로14길,195965,442143,11620,11620585,관악구,낙성대동
100,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,관악구,청룡동
...,...,...,...,...,...,...,...,...,...,...
38082,A,골목상권,1000454,통일로37길,195100,454161,11410,11410620,서대문구,홍제1동
38107,A,골목상권,1000455,통일로39길,194935,454243,11410,11410620,서대문구,홍제1동
38132,A,골목상권,1000457,홍제내길,194424,454124,11410,11410620,서대문구,홍제1동
38157,D,발달상권,1001185,서울 서대문구 신촌역,194426,450896,11410,11410585,서대문구,신촌동


In [42]:
df13 = load_to_df('상권배후지-소득소비')
# df13['hl_소득_구간_코드'].unique()   #5~9
df13.columns

Index(['기준_년_분기_코드', '상권_코드', 'hl_월_평균_소득_금액', 'hl_소득_구간_코드', 'hl_지출_총금액',
       'hl_식료품_지출_총금액', 'hl_의류_신발_지출_총금액', 'hl_생활용품_지출_총금액', 'hl_의료비_지출_총금액',
       'hl_교통_지출_총금액', 'hl_여가_지출_총금액', 'hl_문화_지출_총금액', 'hl_교육_지출_총금액',
       'hl_유흥_지출_총금액'],
      dtype='object')

In [43]:
df13 = df13.merge(map_df, on='상권_코드').drop_duplicates()

In [44]:
df13.columns

Index(['기준_년_분기_코드', '상권_코드', 'hl_월_평균_소득_금액', 'hl_소득_구간_코드', 'hl_지출_총금액',
       'hl_식료품_지출_총금액', 'hl_의류_신발_지출_총금액', 'hl_생활용품_지출_총금액', 'hl_의료비_지출_총금액',
       'hl_교통_지출_총금액', 'hl_여가_지출_총금액', 'hl_문화_지출_총금액', 'hl_교육_지출_총금액',
       'hl_유흥_지출_총금액', '상권_구분_코드', '상권_구분_코드_명', '상권_코드_명', '엑스좌표_값', '와이좌표_값',
       '시군구_코드', '행정동_코드', '시군구_코드_명', '행정동_코드_명'],
      dtype='object')

In [45]:
expense_df = df13[['기준_년_분기_코드','상권_구분_코드_명', '상권_코드_명','시군구_코드_명', '행정동_코드_명',
                   'hl_월_평균_소득_금액', 'hl_소득_구간_코드', 'hl_지출_총금액','hl_식료품_지출_총금액', 'hl_의류_신발_지출_총금액', 'hl_생활용품_지출_총금액', 
                   'hl_의료비_지출_총금액','hl_교통_지출_총금액', 'hl_여가_지출_총금액', 'hl_문화_지출_총금액', 'hl_교육_지출_총금액','hl_유흥_지출_총금액']]

In [46]:
expense_df.head()

,기준_년_분기_코드,상권_구분_코드_명,상권_코드_명,시군구_코드_명,행정동_코드_명,hl_월_평균_소득_금액,hl_소득_구간_코드,hl_지출_총금액,hl_식료품_지출_총금액,hl_의류_신발_지출_총금액,hl_생활용품_지출_총금액,hl_의료비_지출_총금액,hl_교통_지출_총금액,hl_여가_지출_총금액,hl_문화_지출_총금액,hl_교육_지출_총금액,hl_유흥_지출_총금액
0,2019_2,골목상권,풍성로37가길,강동구,성내2동,2836071,6,12120296344,3292343150,1466629217,892634747,1607315609,1972205430,444384439,377360177,1550457530,516966045
1,2019_1,골목상권,풍성로37가길,강동구,성내2동,2923445,6,12120296344,3292343150,1466629217,892634747,1607315609,1972205430,444384439,377360177,1550457530,516966045
2,2019_2,골목상권,천호옛길,강동구,성내2동,3139104,7,8976116500,2526811041,1073907318,659101424,1225903534,1395888645,327727539,283116450,1109489033,374171516
3,2019_1,골목상권,천호옛길,강동구,성내2동,3191004,7,8976116500,2526811041,1073907318,659101424,1225903534,1395888645,327727539,283116450,1109489033,374171516
4,2019_2,골목상권,천호옛14길,강동구,성내2동,2739189,6,6175425196,1846295541,733624837,464028324,882017929,906576672,219745655,205715662,662629513,254791063


In [60]:
df20 = pd.read_csv('data/20.2019_소득분위.csv', encoding='utf-8')

In [62]:
df20.info()
df20['월소득10분위별'].unique()
# df20

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   월소득10분위별     132 non-null    object 
 1   가계수지항목별      132 non-null    object 
 2   전체가구(1분기)    132 non-null    float64
 3   근로자가구(1분기)   132 non-null    float64
 4   근로자외가구(1분기)  132 non-null    float64
 5   전체가구(2분기)    132 non-null    float64
 6   근로자가구(2분기)   132 non-null    float64
 7   근로자외가구(2분기)  132 non-null    float64
dtypes: float64(6), object(2)
memory usage: 8.4+ KB


array(['전체평균', '1분위', '2분위', '3분위', '4분위', '5분위', '6분위', '7분위', '8분위',
       '9분위', '10분위'], dtype=object)

In [66]:
# filter하는 방법중 하나(value에서 찾을때 이용)

targets = ['5분위', '6분위', '7분위', '8분위','9분위']
df20_1=df20[df20['월소득10분위별'].apply(lambda sentence: any(Word in sentence for Word in targets))]
df20_1['월소득10분위별'].unique()
# df20_1

array(['5분위', '6분위', '7분위', '8분위', '9분위'], dtype=object)

In [50]:
df20_2=df20_1.set_index('월소득10분위별')

In [74]:
df20_1  # 월소득10분위별 중 5~9 추출
pop_income=pd.concat([df20_1[df20_1['가계수지항목별']=='소득(원)'],df20_1[df20_1['가계수지항목별']=='근로소득(원)']])
# pop_income.info()
# # pop_income
pop_income.iloc[:,2:]=pop_income.iloc[:,2:].astype('int64')
pop_income.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 63 to 113
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   월소득10분위별     10 non-null     object
 1   가계수지항목별      10 non-null     object
 2   전체가구(1분기)    10 non-null     int64 
 3   근로자가구(1분기)   10 non-null     int64 
 4   근로자외가구(1분기)  10 non-null     int64 
 5   전체가구(2분기)    10 non-null     int64 
 6   근로자가구(2분기)   10 non-null     int64 
 7   근로자외가구(2분기)  10 non-null     int64 
dtypes: int64(6), object(2)
memory usage: 720.0+ bytes


In [109]:
hl_people_df.columns

Index(['기준_년_분기_코드', '상권_코드', 'hl_총_유동인구_수', 'hl_남성_유동인구_수', 'hl_여성_유동인구_수',
       'hl_총_상주인구_수', 'hl_남성_상주인구_수', 'hl_여성_상주인구_수', 'hl_총_가구_수',
       'hl_아파트_가구_수', 'hl_비_아파트_가구_수', 'hl_주중_유동인구_수', 'hl_주말_유동인구_수',
       'hl_총_직장인구_수', 'hl_남성_직장인구_수', 'hl_여성_직장인구_수', '상권_구분_코드', '상권_구분_코드_명',
       '상권_코드_명', '엑스좌표_값', '와이좌표_값', '시군구_코드', '행정동_코드', '시군구_코드_명',
       '행정동_코드_명'],
      dtype='object')

In [110]:
hl_people_df[['기준_년_분기_코드','상권_구분_코드_명', '상권_코드_명','시군구_코드_명', '행정동_코드_명','hl_총_가구_수']]

,기준_년_분기_코드,상권_구분_코드_명,상권_코드_명,시군구_코드_명,행정동_코드_명,hl_총_가구_수
0,2019_1,골목상권,부일로11길,구로구,수궁동,5547
1,2019_2,골목상권,부일로11길,구로구,수궁동,5547
2,2019_1,골목상권,부일로15길,구로구,수궁동,4425
3,2019_2,골목상권,부일로15길,구로구,수궁동,4425
4,2019_1,골목상권,부일로1길,구로구,수궁동,3163
...,...,...,...,...,...,...
2015,2019_2,골목상권,당곡길,관악구,보라매동,11285
2016,2019_1,골목상권,대학14길,관악구,대학동,10033
2017,2019_2,골목상권,대학14길,관악구,대학동,10033
2018,2019_1,골목상권,대학길,관악구,대학동,12309


In [111]:
hl_people_df['hl_총_가구_수'].sum()

15761950

In [123]:
t=hl_people_df[hl_people_df['기준_년_분기_코드']=='2019_1']
t.columns
t['hl_총_가구_수'].sum()

7880975

In [17]:
df0=load_to_df( '상권-영역' )
# df0.head()
# df0.info()
df10 = load_to_df('행정동별 상권변화지표')
# df10.head()
df15=load_to_df( '자치구별 상권변화지표' )
# df15.info()
# df15.head()

ValueError: Length mismatch: Expected axis has 10 elements, new values have 8 elements